# obonet tutorial: import and analyze the Gene Ontology in Python

This tutotial shows:

1. How to read the Gene Ontology OBO export into `networkx` using the [`obonet`](https://github.com/dhimmel/obonet) package.
2. Simple tasks you can do with the `networkx.MultiDiGraph` data structure.

In [1]:
import networkx
import obonet

### Read the Gene Ontology

Learn more about the Gene Ontology (GO) downloads [here](http://geneontology.org/page/download-ontology). Note how we can read the OBO file from a URL. `obonet.read_obo` automically detects whether it's passed a local path, URL, or open file. In addition, `obonet.read_obo` will automtically decompress files ending in `.gz`, `.bz2`, or `.gz`.

In [2]:
%%time
url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
graph = obonet.read_obo(url)

CPU times: user 4.46 s, sys: 209 ms, total: 4.67 s
Wall time: 6.8 s


In [3]:
# Number of nodes
len(graph)

44264

In [4]:
# Number of edges
graph.number_of_edges()

87642

In [5]:
# Check if the ontology is a DAG
networkx.is_directed_acyclic_graph(graph)

True

### Lookup node properties

Returns a dictionary.

In [6]:
# Retreive properties of phagocytosis
graph.nodes['GO:0006909']

{'name': 'phagocytosis',
 'namespace': 'biological_process',
 'def': '"A vesicle-mediated transport process that results in the engulfment of external particulate material by phagocytes and their delivery to the lysosome. The particles are initially contained within phagocytic vacuoles (phagosomes), which then fuse with primary lysosomes to effect digestion of the particles." [ISBN:0198506732]',
 'xref': ['Wikipedia:Phagocytosis'],
 'is_a': ['GO:0016192']}

In [7]:
# Retreive properties of pilus shaft
graph.nodes['GO:0009418']

{'name': 'pilus shaft',
 'namespace': 'cellular_component',
 'def': '"The long, slender, mid section of a pilus." [GOC:jl]',
 'synonym': ['"fimbrial shaft" EXACT []'],
 'is_a': ['GO:0110165'],
 'relationship': ['part_of GO:0009289']}

### Create name mappings

Note that for some OBO ontologies, some nodes only have an id and not a name ([see issue](https://github.com/dhimmel/obonet/issues/11)).

In [8]:
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
name_to_id = {data['name']: id_ for id_, data in graph.nodes(data=True) if 'name' in data}

In [9]:
# Get the name for GO:0042552
id_to_name['GO:0042552']

'myelination'

In [10]:
# Get the id for myelination
name_to_id['myelination']

'GO:0042552'

### Find parent or child relationships

In [11]:
# Find edges to parent terms
node = name_to_id['pilus']
for child, parent, key in graph.out_edges(node, keys=True):
    print(f'• {id_to_name[child]} ⟶ {key} ⟶ {id_to_name[parent]}')

• pilus ⟶ is_a ⟶ cell projection


In [12]:
# Find edges to children terms
node = name_to_id['pilus']
for parent, child, key in graph.in_edges(node, keys=True):
    print(f'• {id_to_name[child]} ⟵ {key} ⟵ {id_to_name[parent]}')

• pilus ⟵ part_of ⟵ pilus shaft
• pilus ⟵ part_of ⟵ pilus tip
• pilus ⟵ is_a ⟵ type IV pilus


### Find all superterms of myelination

In [13]:
sorted(id_to_name[superterm] for superterm in networkx.descendants(graph, 'GO:0042552'))

['anatomical structure development',
 'axon ensheathment',
 'biological_process',
 'cellular process',
 'developmental process',
 'ensheathment of neurons',
 'multicellular organism development',
 'multicellular organismal process',
 'nervous system development',
 'system development']

### Find all subterms of myelination

In [14]:
sorted(id_to_name[subterm] for subterm in networkx.ancestors(graph, 'GO:0042552'))

['central nervous system myelin formation',
 'central nervous system myelin maintenance',
 'central nervous system myelination',
 'myelin assembly',
 'myelin maintenance',
 'myelination in peripheral nervous system',
 'myelination of anterior lateral line nerve axons',
 'myelination of lateral line nerve axons',
 'myelination of posterior lateral line nerve axons',
 'negative regulation of myelination',
 'paranodal junction assembly',
 'peripheral nervous system myelin formation',
 'peripheral nervous system myelin maintenance',
 'positive regulation of myelination',
 'regulation of myelination']

### Find all paths to the root

In [15]:
paths = networkx.all_simple_paths(
    graph,
    source=name_to_id['starch binding'],
    target=name_to_id['molecular_function']
)
for path in paths:
    print('•', ' ⟶ '.join(id_to_name[node] for node in path))

• starch binding ⟶ polysaccharide binding ⟶ carbohydrate binding ⟶ binding ⟶ molecular_function


### See the ontology metadata

In [16]:
graph.graph

{'name': 'go',
 'typedefs': [{'id': 'negatively_regulates',
   'name': 'negatively regulates',
   'namespace': 'external',
   'xref': ['RO:0002212'],
   'is_a': ['regulates']},
  {'id': 'part_of',
   'name': 'part of',
   'namespace': 'external',
   'xref': ['BFO:0000050'],
   'is_transitive': 'true'},
  {'id': 'positively_regulates',
   'name': 'positively regulates',
   'namespace': 'external',
   'xref': ['RO:0002213'],
   'holds_over_chain': ['negatively_regulates negatively_regulates'],
   'is_a': ['regulates']},
  {'id': 'regulates',
   'name': 'regulates',
   'namespace': 'external',
   'xref': ['RO:0002211'],
   'is_transitive': 'true'},
  {'id': 'term_tracker_item',
   'name': 'term tracker item',
   'namespace': 'external',
   'xref': ['IAO:0000233'],
   'is_metadata_tag': 'true',
   'is_class_level': 'true'}],
 'instances': [],
 'format-version': '1.2',
 'data-version': 'releases/2020-10-09',
 'subsetdef': ['chebi_ph7_3 "Rhea list of ChEBI terms representing the major specie

## Create a dictionary of obsolete terms to their replacements

In [17]:
graph_with_obs = obonet.read_obo(url, ignore_obsolete=False)
len(graph_with_obs)

47281

In [18]:
old_to_new = dict()
for node, data in graph_with_obs.nodes(data=True):
    for replaced_by in data.get("replaced_by", []):
        old_to_new[node] = replaced_by
list(old_to_new.items())[:5]

[('GO:0000108', 'GO:0000109'),
 ('GO:0000174', 'GO:0000750'),
 ('GO:0000260', 'GO:0046961'),
 ('GO:0000261', 'GO:0046962'),
 ('GO:0000284', 'GO:0000753')]